# Portfolio Workflow

In [1]:
import logging
import logging.config

logging.config.fileConfig('./config/logging.ini')
logger = logging.getLogger('GenerateBacktest')

import configparser
from platform import python_version
from pathlib import Path

# Set the import path for the project tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, 'tools')

# Project imports
import importlib
import trading_factors_yahoo as alpha_factors
importlib.reload(alpha_factors)
import utils
importlib.reload(utils)
import nonoverlapping_estimator as ai_estimator
importlib.reload(ai_estimator)

import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pickle

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

logger.info(f'Python version: {python_version()}')
logger.info(f'Pandas version: {pd.__version__}')

2022-05-15 19:50:03,045|numexpr.utils|INFO|NumExpr defaulting to 4 threads.
Sci-Kit version: 0.24.1
Sci-Kit version: 0.24.1
2022-05-15 19:50:06,668|GenerateBacktest|INFO|Python version: 3.8.8
2022-05-15 19:50:06,668|GenerateBacktest|INFO|Pandas version: 1.3.5


In [2]:
config = configparser.ConfigParser()
config.read('./config/config.ini')
default_config = config['BackTest']

# Price History data and Alphs Factors

In [3]:
price_histories_file_name = default_config['DataDirectory'] + '/' + default_config['PriceHistoriesFileName']
ai_alpha_factors_file_name = default_config['DataDirectory'] + '/' + default_config['AIAlphaFileName']
beta_factors_file_name = default_config["DataDirectory"] + '/' + default_config['BetaFactorsFileName']


logger.info(f'PRICE_HISTORIES_FILE|{price_histories_file_name}...')
price_histories = pd.read_csv(price_histories_file_name, header=[0, 1], index_col=[0], parse_dates=True, low_memory=False)
pricing = price_histories.Close
print(f'You have {len(pricing.columns)} stocks from picing')

logger.info(f'ALPHA_VECTORS_FILE|{ai_alpha_factors_file_name}...')
alpha_vectors = pd.read_csv(ai_alpha_factors_file_name, parse_dates=['Date']).set_index(['Date']).sort_index()
logger.info(f'ALPHA_VECTORS_STOCKS|{len(alpha_vectors.columns)}')

logger.info(f'BEATA_FACTORS_FILE|{beta_factors_file_name}...')
with open(beta_factors_file_name, 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    daily_betas = pickle.load(f)
logger.info(f'DAILY_BETAS|{len(daily_betas)}')

risk_cap = float(default_config['risk_cap'])
weights_max = float(default_config['weights_max'])
weights_min = float(default_config['weights_min'])

logger.info(f'OPT|risk_cap|{risk_cap}')
logger.info(f'OPT|weights_max|{weights_max}')
logger.info(f'OPT|weights_min|{weights_min}')

2022-05-15 19:50:06,725|GenerateBacktest|INFO|PRICE_HISTORIES_FILE|./data/price_histories_yahoo.csv...
You have 501 stocks from picing
2022-05-15 19:50:07,768|GenerateBacktest|INFO|ALPHA_VECTORS_FILE|./data/pre_backtest_alpha_vectors.csv...
2022-05-15 19:50:07,935|GenerateBacktest|INFO|ALPHA_VECTORS_STOCKS|501
2022-05-15 19:50:07,935|GenerateBacktest|INFO|BEATA_FACTORS_FILE|./data/daily_beta.pickle...
2022-05-15 19:50:09,684|GenerateBacktest|INFO|DAILY_BETAS|1009
2022-05-15 19:50:09,684|GenerateBacktest|INFO|OPT|risk_cap|0.15
2022-05-15 19:50:09,684|GenerateBacktest|INFO|OPT|weights_max|0.125
2022-05-15 19:50:09,684|GenerateBacktest|INFO|OPT|weights_min|0.0


# Back test AI Alpha and Daily Betas to produce optimal weights

In [ ]:
import portfolio_optimizer
from portfolio_optimizer import OptimalHoldings
importlib.reload(portfolio_optimizer)

returns = alpha_factors.FactorReturns(price_histories).factor_data
dlyreturn_n_days_delay = 5
delayed_returns = returns[-252:].shift(-dlyreturn_n_days_delay).dropna()
start_date = list(delayed_returns.index)[0]
end_date = list(delayed_returns.index)[-1]
logger.info(f'OPT|{start_date}|{end_date}')
current_holdings = pd.DataFrame(np.zeros(len(delayed_returns.columns)), index=delayed_returns.columns)
init_port_value = portfolio_value = float(default_config['init_port_value'])
min_viable_port_return = float(default_config['min_viable_port_return'])
logger.info(f'OPT|INIT_PORT_VALUE|{init_port_value}')
portfolio_growth = {}
for opt_date in tqdm(delayed_returns.index.to_list()[-252::dlyreturn_n_days_delay], desc='Dates', unit=' Portfolio Optimization'):
    alpha_vector = pd.DataFrame(alpha_vectors.loc[opt_date])
    risk_model = daily_betas[opt_date.strftime('%m/%d/%Y')]
    est_return = delayed_returns.loc[opt_date]
    optimal_weights = OptimalHoldings(risk_cap=risk_cap,weights_max=weights_max, weights_min=weights_min).find(alpha_vector, risk_model.factor_betas_, risk_model.factor_cov_matrix_, risk_model.idiosyncratic_var_vector_)
    long_weights = optimal_weights[(100 * optimal_weights['optimalWeights']).round() > 0]
    long_holdings = (long_weights['optimalWeights'] * portfolio_value).round(0)
    new_holdings = long_holdings + (long_holdings * est_return[long_holdings.index])
    portfolio_value = new_holdings.sum()
    portfolio_growth[opt_date] = portfolio_value
    current_holdings = new_holdings

port_return = round(np.log(portfolio_value / init_port_value) * 100, 2)
logger.info(f'OPT|INIT_PORT_VALUE|{init_port_value}|FINAL_PORT_VALUE|{portfolio_value}|PORT_RETURN|{port_return}%')
if port_return >= min_viable_port_return:
    logger.info(f'OPT|PROCEED|{port_return}')
else:
    logger.warn(f'OPT|STOP|{port_return}')
    raise RuntimeError(f'Backtest indicates this strategy needs more work! ({port_return})') from None
    
pd.Series(portfolio_growth).plot()

2022-05-15 19:50:09,760|GenerateBacktest|INFO|OPT|2021-05-17 00:00:00|2022-05-06 00:00:00
2022-05-15 19:50:09,761|GenerateBacktest|INFO|OPT|INIT_PORT_VALUE|100000.0


Dates:   0%|          | 0/50 [00:00<?, ? Portfolio Optimization/s]

In [ ]:
port_return = round(np.log(portfolio_value / init_port_value) * 100, 2)
logger.info(f'OPT|INIT_PORT_VALUE|{init_port_value}|FINAL_PORT_VALUE|{portfolio_value}|PORT_RETURN|{port_return}%')

In [ ]:
pd.Series(portfolio_growth).plot()

In [ ]:
port_return = round(np.log(portfolio_value / init_port_value) * 100, 2)
print(f'Starting portfolio: {init_port_value} Ending portfolio: {portfolio_value} Return: {port_return}%')
if port_return >= 8:
    print('Backtest indicates its okay to proceed with this strategy.')
else:
    raise RuntimeError(f'Backtest indicates this strategy needs more work! ({port_return})') from None